In [2]:
from requests_html import HTMLSession
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import re

In [3]:
multibuy_pattern_deal = re.compile(r"(\d+ x \w+)")
multibuy_pattern_trees = re.compile(r"(\d+ Tree)", flags=re.IGNORECASE)
multibuy_pattern_pack = re.compile(r"(\d+ Pack)", flags=re.IGNORECASE)
multibuy_pattern_plant = re.compile(r"(\d+ Plant)", flags=re.IGNORECASE)

numeric_pattern_compiled = re.compile(r"(\d+)")


def extract_quantity_from_text(product_text: str) -> tuple[int, bool]:
    multibuy_str = re.search(multibuy_pattern_deal, product_text) or re.search(multibuy_pattern_trees, product_text) or re.search(multibuy_pattern_pack, product_text) or re.search(multibuy_pattern_plant, product_text)
    if multibuy_str:
        quantity = int(
            re.search(numeric_pattern_compiled, multibuy_str.group(0)).group(0)
        )
        multibuy = True
    else:
        quantity = 1
        multibuy = False
    return quantity, multibuy


size_pattern_litres = re.compile(r"(0?\.?\d+\s?L)")
size_pattern_centimetres = re.compile(r"(\d+\s?-\s?\d+\s?cm)", flags=re.IGNORECASE)
size_pattern_centimetres = re.compile(r"(\d+\s?-\s?\d+\s?cm)", flags=re.IGNORECASE)


def extract_size_from_text(text: str) -> str:
    size_str = re.search(size_pattern_litres, text)
    if size_str:
        size = size_str.group(0).replace(" L", "L")
    else:
        size_str = re.search(size_pattern_centimetres, text)
        if size_str:
            size = size_str.group(0)
        else:
            if "tree" in text.lower():
                return "whip"
            else:
                print(text)
                raise Exception("size is wrong")
    return size


def extract_price_from_text(price_str):
    return float(re.sub(r"[^\d.\.]", "", price_str))

In [6]:
def parse_products(URL: str, category: str, products: list) -> list[dict]:
    results = []
    # Skip the first row as that as the heading though its not labeled as such
    for product in products[1:]:
        try:
            product_url = product.find("a")['href']
            product_url = urljoin(URL, product_link["href"])
            product_name = product.find("div", class_="botanical-name").text
            image = product.find("img", class_="imageWrap").find("img")
            if image is None:
                img_url = None
            else:
                img_url = image["src"]
            description = product.find("div", class_="product description product-item-description").text
            price_inc_vat = extract_price_from_text(product.find("span", class_="price").text)

            misc = content.find_all("div", class_ = "amlabel-text")
            for entry in misc:
                if roots in (entry.text.to_ == 
            
            
           
            size = product.find("div", class_="columnField").find("span").text
            try:
                stock = int(product.find("div", class_="inStock").text)
            except ValueError:
                stock = 0
            results.append(
                {
                    "source": "tullys",
                    "source_url": URL,
                    "product_url": product_url,
                    "category": category,
                    "product_name": product_name,
                    "img_url": img_url,
                    "description": description,
                    "price": price_inc_vat,
                    "size": size,
                    "stock": stock,
                    "quantity": quantity
                }
            )
        except Exception as error:
            template = "An exception of type {0} occurred. Arguments:\n{1!r}"
            message = template.format(type(error).__name__, error.args)
            print(message)
    return results



def parse_url(URL: str, category: str) -> list[dict]:

    session = HTMLSession()
    page_number = 1
    results = []

    while (page := session.get(f"{URL}?p={page_number}")).status_code == 200:
        
        content = BeautifulSoup(page.content, "html.parser")
        if content.find("div", class_="message info empty"):
            break
        print(f"page: {page_number}")
        products = content.find(class_ = "products list items product-items").find_all("li")
        results.extend(parse_products(URL=URL, category=category, products=products))
        
            


        page_number += 1    

    print(f"Found {len(results)} products for {category}")
    return results                           

URL = "https://www.gardens4you.ie/garden-plants/perennials/all-perennials"
category = "Perennials"
results = parse_url(URL, category)
for result in results:
    print("\n".join("{!r}: {!r},".format(k, v) for k, v in result.items()) )

page: 1
https://www.gardens4you.ie/garden-plants/perennials/all-perennials/acaena-buchananii-a12449.html
https://www.gardens4you.ie/garden-plants/perennials/all-perennials/acanthus-mollis-bears-breeches-a07855.html
https://www.gardens4you.ie/garden-plants/perennials/all-perennials/achillea-milefolium-summer-pastels-a11425.html
https://www.gardens4you.ie/garden-plants/perennials/all-perennials/achillea-millefolium-pink-x-3-a00780.html
https://www.gardens4you.ie/garden-plants/perennials/all-perennials/achillea-millefolium-red-x-3-a09571.html
https://www.gardens4you.ie/garden-plants/perennials/all-perennials/achillea-millefolium-terracotta-9-cm-pot-a14684.html
https://www.gardens4you.ie/garden-plants/perennials/all-perennials/achillea-millefolium-yellow-x-3-a09572.html
https://www.gardens4you.ie/garden-plants/perennials/all-perennials/achillea-tomentosa-aurea-9-cm-pot-a14686.html
https://www.gardens4you.ie/garden-plants/perennials/all-perennials/aconitum-carmichaelii-arendsii-x-3-a02113.h